# TensorFlow1 pb 推理

参考：[migrating_checkpoints](https://www.tensorflow.org/guide/migrate/migrating_checkpoints)

下面以模型 [resnet_v2_50](http://download.tensorflow.org/models/resnet_v2_50_2017_04_14.tar.gz) 为例展示。

需要克隆项目 [models](https://github.com/tensorflow/models)，然后执行如下操作。

In [1]:
import tensorflow as tf
try:
    tf1 = tf.compat.v1
except (ImportError, AttributeError):
    tf1 = tf

2023-06-15 20:35:27.704544: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-15 20:35:27.751463: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-15 20:35:27.752582: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-15 20:35:28.683585: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


切换到 `models/research/slim` 目录下：

In [2]:
%cd /media/pc/data/lxw/ai/tasks/models/research/slim

/media/pc/data/lxw/ai/tasks/models/research/slim


将 ckpt 模型转换为 pb 模型：

In [3]:
import tensorflow as tf
try:
    tf1 = tf.compat.v1
except (ImportError, AttributeError):
    tf1 = tf
from PIL import Image
import numpy as np
from nets import resnet_v2
from preprocessing.preprocessing_factory import get_preprocessing
import tf_slim as slim
import shutil

def remove_dir(path):
    try:
        shutil.rmtree(path)
    except:
        ...
# from tensorflow.python.platform import gfile
# from tensorflow.python.tools import freeze_graph
tf.get_logger().setLevel('ERROR')
preprocessing = get_preprocessing("resnet_v2_50")
image_size = 224
path = '/media/pc/data/board/arria10/lxw/data/test/cat.png' # 将要预测的图片路径
checkpoints_path = "/media/pc/data/board/arria10/lxw/tests/npu_user_demos/models/resnet50_v2_tf/weight/resnet_v2_50.ckpt"
with Image.open(path) as im:
    if im.mode != "RGB":
        im.convert("RGB")
    image = np.asarray(im)
# 预处理

model_dir = 'temp/resnet_v2_50'
remove_dir(model_dir)
with tf1.Graph().as_default() as graph:
    image = tf.constant(image)
    processed_image = preprocessing(image, image_size, image_size)
    _processed_images  = tf.expand_dims(processed_image/256, 0)
    with tf1.Session() as sess:
        np_processed_images = sess.run(_processed_images)
    processed_images = tf.constant(np_processed_images)
    # 创建模型时，使用默认的参数范围（arg scope）来配置批归一化（batch norm）参数。
    with slim.arg_scope(resnet_v2.resnet_arg_scope()):
        logits, end_points = resnet_v2.resnet_v2_50(processed_images, num_classes=1001,
                                                    global_pool=True,
                                                    is_training=False)
    probabilities = tf.nn.softmax(logits)
    variables = slim.get_model_variables('resnet_v2_50')
    init_fn = slim.assign_from_checkpoint_fn(checkpoints_path, variables)
    with tf1.Session() as sess:
        init_fn(sess)
        # np_probabilities, np_processed_images = sess.run([probabilities, processed_images])
        np_probabilities = sess.run(probabilities)
        tf1.saved_model.simple_save(
            sess, model_dir,
            inputs={'inputs': processed_images},
            outputs={'output': probabilities}
        )

2023-06-15 20:35:30.434574: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2023-06-15 20:35:30.500596: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:353] MLIR V1 optimization pass is not enabled
/media/pc/data/tmp/cache/conda/envs/tvmz/lib/python3.10/site-packages/tensorflow/python/keras/engine/base_layer_v1.py:1694: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '


加载保存的模型：

In [4]:
import tensorflow as tf
try:
    tf1 = tf.compat.v1
except (ImportError, AttributeError):
    tf1 = tf
from PIL import Image
import numpy as np
tf.get_logger().setLevel('ERROR')

def preprocess_image(image, output_height, output_width):
    with tf1.Graph().as_default() as graph:
        image = tf.constant(image)
        processed_image = preprocessing(image, output_height, output_width)
        _processed_image  = processed_image/256
        with tf1.Session() as sess:
            np_processed_image = sess.run(_processed_image)
    return np_processed_image

image_size = 224
path = '/media/pc/data/board/arria10/lxw/data/test/cat.png' # 将要预测的图片路径
with Image.open(path) as im:
    if im.mode != "RGB":
        im.convert("RGB")
    image = np.asarray(im)
# 预处理

model_dir = 'temp/resnet_v2_50'
# remove_dir(model_dir)
np_processed_image = preprocess_image(image, image_size, image_size)
np_processed_images = np.expand_dims(np_processed_image, axis=0)
# 加载模型
loaded_model = tf.saved_model.load(model_dir)
loaded_model = loaded_model.signatures[tf.saved_model.DEFAULT_SERVING_SIGNATURE_DEF_KEY]
out = loaded_model(tf.constant(np_processed_images))['output'].numpy()

打印标签信息：

In [5]:
from github import Github


g = Github(user_agent="xinetzone")
repo = g.get_repo("tensorflow/models")
label_content = repo.get_contents("research/slim/datasets/imagenet_lsvrc_2015_synsets.txt")
imagenet_labels = label_content.decoded_content.decode().split()
assert len(imagenet_labels) == 1000
metadata = repo.get_contents("research/slim/datasets/imagenet_metadata.txt")
imagenet_metadata = metadata.decoded_content.decode().splitlines()
synset_to_human = {}
for metadata in imagenet_metadata:
    name, value = metadata.split("\t")
    synset_to_human[name] = value
name2id = {name: k+1 for k, name in enumerate(imagenet_labels)}

topk = 5
sorted_inds = out[0].argsort()[::-1]
for sorted_ind in sorted_inds[:topk]:
    label = synset_to_human[imagenet_labels[sorted_ind-1]]
    print(f"{sorted_ind-1}: {label.ljust(20)}\t{out[0][sorted_ind-1]}")

282: tiger cat           	0.15822742879390717
285: Egyptian cat        	0.0009194915182888508
281: tabby, tabby cat    	9.054027759702876e-05
278: kit fox, Vulpes macrotis	0.0044543202966451645
277: red fox, Vulpes vulpes	2.409669548342208e-07
